# 🧪 AI-Powered Drug Discovery using Deep Learning
This notebook implements a novel deep learning model for predicting drug-target interactions using SMILES representations and deep neural networks.

**Includes:**
- Data loading & preprocessing
- Molecular fingerprinting using RDKit
- PyTorch model
- Cross-validation
- Dashboard-ready visualizations

In [12]:
%pip install pandas numpy scikit-learn matplotlib seaborn rdkit-pypi torch


Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import (
    accuracy_score, roc_auc_score, f1_score,
    precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay
)
from rdkit import Chem
from rdkit.Chem import AllChem
import matplotlib.pyplot as plt

Note: you may need to restart the kernel to use updated packages.



A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\USER HP\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\USER HP\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\USER HP\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\USER HP\AppData\Local\Programs\Python\Python310\lib\site-packag

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\USER HP\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\USER HP\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\USER HP\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\USER HP\AppData\Local\Programs\Python\Python310\lib\site-packag

AttributeError: _ARRAY_API not found

...

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\USER HP\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\USER HP\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\USER HP\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\USER HP\AppData\Local\Programs\Python\Python310\lib\site-packag

AttributeError: _ARRAY_API not found


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "c:\Users\USER HP\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "c:\Users\USER HP\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "c:\Users\USER HP\AppData\Local\Programs\Python\Python310\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "c:\Users\USER HP\AppData\Local\Programs\Python\Python310\lib\site-packag

AttributeError: _ARRAY_API not found

In [14]:

df = pd.read_csv("drug_target_sample_dataset.csv")


In [15]:


# Convert SMILES to Morgan Fingerprints
def smiles_to_fp(smiles):
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        return np.zeros(2048)
    return np.array(AllChem.GetMorganFingerprintAsBitVect(mol, 2, nBits=2048))

df['fingerprint'] = df['SMILES'].apply(smiles_to_fp)
X = np.stack(df['fingerprint'].values)
y = df['Label'].values



In [16]:
# ---------------------------
# Define Neural Network
# ---------------------------
class DrugClassifier(nn.Module):
    def __init__(self):
        super(DrugClassifier, self).__init__()
        self.fc1 = nn.Linear(2048, 512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

In [17]:
# ---------------------------
# Training + Cross Validation
# ---------------------------
kf = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
accuracies, aucs, f1s, precisions, recalls = [], [], [], [], []

for fold, (train_idx, test_idx) in enumerate(kf.split(X, y)):
    model = DrugClassifier()
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.BCELoss()

    X_train = torch.FloatTensor(X[train_idx])
    y_train = torch.FloatTensor(y[train_idx]).view(-1, 1)
    X_test = torch.FloatTensor(X[test_idx])
    y_test = torch.FloatTensor(y[test_idx]).view(-1, 1)

    # Training Loop
    for epoch in range(100):
        model.train()
        optimizer.zero_grad()
        output = model(X_train)
        loss = criterion(output, y_train)
        loss.backward()
        optimizer.step()

    # Evaluation
    model.eval()
    with torch.no_grad():
        probs = model(X_test).numpy()
        preds = (probs >= 0.5).astype(int)

        acc = accuracy_score(y_test, preds)
        auc = roc_auc_score(y_test, probs)
        f1 = f1_score(y_test, preds)
        precision = precision_score(y_test, preds)
        recall = recall_score(y_test, preds)

        cm = confusion_matrix(y_test, preds)
        disp = ConfusionMatrixDisplay(confusion_matrix=cm)
        disp.plot()
        plt.title(f"Confusion Matrix - Fold {fold+1}")
        plt.show()

        # Store Metrics
        accuracies.append(acc)
        aucs.append(auc)
        f1s.append(f1)
        precisions.append(precision)
        recalls.append(recall)

        print(f"Fold {fold+1} | Accuracy: {acc:.4f}, AUC: {auc:.4f}, "
              f"F1: {f1:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}")

# ---------------------------
# Final Metric Summary
# ---------------------------
print("\n📊 Average Metrics Across 3 Folds:")
print(f"Accuracy  : {np.mean(accuracies):.4f}")
print(f"AUC       : {np.mean(aucs):.4f}")
print(f"F1 Score  : {np.mean(f1s):.4f}")
print(f"Precision : {np.mean(precisions):.4f}")
print(f"Recall    : {np.mean(recalls):.4f}")

NameError: name 'f1_score' is not defined